In [ ]:
import pandas as pd
import re

In [ ]:
# Define the path to the CSV file
orig_file_path = '/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/supplier_name_zip_code orig.csv'
new_file_path = '/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/supplier_name_zip_code.csv'

In [ ]:
# Read the CSV file into a pandas DataFrame
try:
    df = pd.read_csv(orig_file_path)
    print(f"Successfully read {orig_file_path}")
    print(f"Initial number of rows: {len(df)}")
except FileNotFoundError:
    print(f"Error: File not found at {orig_file_path}")
    df = None

In [ ]:
# Define the regex for valid US ZIP codes
# Note: The file can contain 4 digit zip codes. In this case, we add a 0 at the beginning
zip_code_regex = re.compile(r'^\d{4}|\d{5}(-\d{4})?$')


In [ ]:
# Ensure the DataFrame was loaded and the zip code column exists
if df is not None and 'Supplier_Zip_Postal_Code' in df.columns:
    # Convert the zip code column to string type to handle potential mixed types and NaN values
    df['Supplier_Zip_Postal_Code'] = df['Supplier_Zip_Postal_Code'].astype(str)
    
    # Filter out rows where the zip code does not match the regex
    # Keep rows where the zip code is valid or is NaN/None (handled by astype(str) -> 'nan')
    # We explicitly check for 'nan' string as well, in case NaNs were present
    initial_rows = len(df)
    df_filtered = df[df['Supplier_Zip_Postal_Code'].apply(lambda x: bool(re.match(zip_code_regex, x)) if pd.notna(x) and x != 'nan' else False)].copy()
    rows_removed = initial_rows - len(df_filtered)
    print(f"Rows removed due to invalid ZIP code format: {rows_removed}")
    print(f"Rows remaining after filtering: {len(df_filtered)}")
else:
    print("DataFrame not loaded or 'Supplier_Zip_Postal_Code' column not found.")
    df_filtered = pd.DataFrame() # Assign empty DataFrame to avoid errors later

In [ ]:
# Remove duplicate rows based on all columns
if not df_filtered.empty:
    initial_rows = len(df_filtered)
    df_deduplicated = df_filtered.drop_duplicates()
    # Count the number of rows removed
    rows_removed = initial_rows - len(df_deduplicated)
    print(f"Rows removed due to duplicates: {rows_removed}")
    print(f"Rows remaining after deduplication: {len(df_deduplicated)}")
else:
    print("Skipping deduplication as the DataFrame is empty.")
    df_deduplicated = pd.DataFrame()

In [ ]:
# Sort the DataFrame alphabetically by zip code
if not df_deduplicated.empty:
    df_sorted = df_deduplicated.sort_values(by='Supplier_Zip_Postal_Code')
    print(f"DataFrame sorted by 'Supplier_Zip_Postal_Code'.")
else:
    print("Skipping sorting as the DataFrame is empty.")
    df_sorted = pd.DataFrame()

In [ ]:
# Display the first few rows of the cleaned and sorted DataFrame
if not df_sorted.empty:
    print("\nFirst 5 rows of the cleaned data:")
    print(df_sorted.head())
else:
    print("Cleaned DataFrame is empty.")

In [ ]:
# Save the cleaned DataFrame back to the original CSV file, overwriting it
if not df_sorted.empty:
    try:
        df_sorted.to_csv(new_file_path, index=False)
        print(f"\nSuccessfully saved cleaned data to {new_file_path}")
        print(f"Final number of rows: {len(df_sorted)}")
    except Exception as e:
        print(f"\nError saving file: {e}")
else:
    print("\nSkipping save operation as the final DataFrame is empty.")

In [ ]:
# Compare the two files :
# -  /Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/supplier_name_zip_code.csv
# -  /Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/supplier_name_zip_code orig.csv

# determine if any zip codes are missing in the cleaned file
try:
    df_orig = pd.read_csv(orig_file_path)
    print(f"Successfully read original file")
    print(f"Initial number of rows: {len(df_orig)}")
except FileNotFoundError:
    print(f"Error: Original file not found.")
    df_orig = None 

# Ensure the original DataFrame was loaded and the zip code column exists
if df_orig is not None and 'Supplier_Zip_Postal_Code' in df_orig.columns:
    # Convert the zip code column to string type to handle potential mixed types and NaN values
    df_orig['Supplier_Zip_Postal_Code'] = df_orig['Supplier_Zip_Postal_Code'].astype(str)
    
    # Find missing zip codes in the cleaned DataFrame
    missing_zip_codes = df_orig[~df_orig['Supplier_Zip_Postal_Code'].isin(df_sorted['Supplier_Zip_Postal_Code'])]

    # remove canadian zip codes
    missing_zip_codes = missing_zip_codes[~missing_zip_codes['Supplier_Zip_Postal_Code'].str.match(r'^[A-Za-z]\d[A-Za-z] \d[A-Za-z]\d')]

    # keep unique values
    missing_zip_codes = missing_zip_codes.drop_duplicates(subset=['Supplier_Zip_Postal_Code'])

    if not missing_zip_codes.empty:
        print(f"\nMissing ZIP codes in cleaned data:")
        print(missing_zip_codes)
    else:
        print("\nNo missing ZIP codes found in cleaned data.")

# Check if the cleaned DataFrame has any zip codes that were not in the original
if df_sorted is not None and 'Supplier_Zip_Postal_Code' in df_sorted.columns:
    extra_zip_codes = df_sorted[~df_sorted['Supplier_Zip_Postal_Code'].isin(df_orig['Supplier_Zip_Postal_Code'])]
    if not extra_zip_codes.empty:
        print(f"\nExtra ZIP codes in cleaned data:")
        print(extra_zip_codes)
    else:
        print("\nNo extra ZIP codes found in cleaned data.")

